<a href="https://colab.research.google.com/github/RandaAlshehri/Bankruptcies/blob/main/reason.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install -U openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.7/767.7 kB 23.5 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [ ]:
!pip install openai scikit-learn pandas

# Classifier

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# Load the dataset
df = pd.read_csv("newDataset.csv")



# Separate features and target
X = df.drop(columns=["Customer_engagement_level"])   # Features
y = df["Customer_engagement_level"]                  # Target (0, 1, 2)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify=y)

# Initialize and train classifier
model = RandomForestClassifier(
    n_estimators=205,
    min_samples_split=10,
    min_samples_leaf=7,
    class_weight="balanced",
)

model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("✅ Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=["Dormant", "Low Engagement", "Active"]))

print("\n📊 Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

import pickle

# Save the trained model to a file
with open("classificationModel.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved successfully!")


✅ Classification Report:

                precision    recall  f1-score   support

       Dormant       0.92      0.89      0.91      1000
Low Engagement       0.82      0.85      0.84      1000
        Active       0.92      0.93      0.92      1000

      accuracy                           0.89      3000
     macro avg       0.89      0.89      0.89      3000
  weighted avg       0.89      0.89      0.89      3000


📊 Confusion Matrix:
[[889 111   0]
 [ 73 851  76]
 [  0  75 925]]


In [ ]:

import numpy as np

importances = model.feature_importances_
feature_names = X.columns


feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})


top_features = feature_importance_df.sort_values(by='Importance', ascending=False).head(5)

print("\n Top 5 Important Features:")
print(top_features.to_string(index=False))



 Top 5 Important Features:
                  Feature  Importance
             credit_score    0.516355
   loyalty_points_balance    0.152591
     app_opens_last_7days    0.114687
missed_payments_last_year    0.060601
   avg_payment_delay_days    0.034602


# Generate_reason

In [ ]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('Secret_Key')



In [ ]:
from openai import OpenAI

In [ ]:
clint = OpenAI()

In [ ]:

dormant_customers = df[df["Customer_engagement_level"] == 0].copy()


# ========================
def generate_reason_prompt_arabic(row):
    prompt = f"""
أنت خبير تحليل بيانات العملاء في القطاع المصرفي.

العميل التالي تم تصنيفه على أنه "خامل". بناءً على خصائصه أدناه، اكتب تحليلًا مهنيًا باللغة العربية يفسر بشكل منطقي سبب خمول هذا العميل. لا تكتفِ بالوصف، بل اربط البيانات ببعضها لتقديم تحليل استنتاجي.

مواصفات العميل:
- عمر الحساب: {row['account_age_days']} يوم
- الحد الائتماني: {row['credit_limit']} ريال سعودي
- نوع الحساب: {row['account_type']}
- متوسط قيمة العمليات: {row['avg_transaction_value']}
- عدد الشكاوى: {row['num_complaints']}
- عدد مكالمات الدعم: {row['calls_to_support']}
- فتح التطبيق آخر ٧ أيام: {row['app_opens_last_7days']}
- إشعارات مفعلة: {"نعم" if row['push_notifications_enabled'] == 1 else "لا"}
- رصيد نقاط الولاء: {row['loyalty_points_balance']}
- عدد الدفعات التلقائية: {row['num_auto_payments']}
- عدد التأخيرات في الدفع: {row['missed_payments_last_year']}
- درجة الائتمان: {row['credit_score']}

قدم تحليلك في فقرة واحدة باللغة العربية.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "أنت مساعد محلل بيانات في بنك."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.5
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ خطأ: {str(e)}"

dormant_customers["arabic_reason"] = dormant_customers.apply(generate_reason_prompt_arabic, axis=1)




In [ ]:
dormant_customers.to_csv("dormant_customers_with_reasons.csv", index=False)
print("\nSample Dormant Customers with Reasons:")
print(dormant_customers[["account_age_days", "credit_limit", "account_type", "arabic_reason"]].head())


Sample Dormant Customers with Reasons:
    account_age_days  credit_limit  account_type  \
0               3204         20000             0   
10              1268          5000             0   
15              3201         10000             3   
18              1715          5000             1   
19              3410         50000             3   

                                        arabic_reason  
0   يتضح من البيانات المقدمة أن العميل يعتبر "خامل...  
10  العميل الذي تم تصنيفه كخامل يمتلك حساباً قديما...  
15  يمكن تفسير خمول العميل من خلال تحليل البيانات ...  
18  يتضح من البيانات المقدمة أن العميل يعتبر "خامل...  
19  يتضح من خلال تحليل بيانات العميل أنه يعتبر "خا...  


In [ ]:
import os
import pandas as pd

# ===  Check if saved file exists before generating ===
csv_path = "dormant_customers_with_reasons.csv"

if os.path.exists(csv_path):
    dormant_customers = pd.read_csv(csv_path)
    print("✅ Loaded GPT explanations from file.")
else:
    print(" Generating GPT-based reasons for dormant customers...")
    dormant_customers["arabic_reason"] = dormant_customers.apply(generate_reason_prompt_arabic, axis=1)

    # Save results as if it's a model output
    dormant_customers.to_csv(csv_path, index=False)
    print(f"Saved GPT explanations to: {csv_path}")


✅ Loaded GPT explanations from file.


In [ ]:
dormant_customers.to_excel("gpt_model_output_v1.xlsx", index=False)


In [ ]:
dormant_customers.to_csv("gpt_model_output_v1.csv", index=False)


In [ ]:
print(dormant_customers[["account_age_days", "credit_limit", "account_type", "arabic_reason"]])

    account_age_days  credit_limit  account_type  \
0               3204         20000             0   
10              1268          5000             0   

                                        arabic_reason  
0   يتضح من البيانات المقدمة أن العميل يعتبر "خامل...  
10  العميل الذي تم تصنيفه كخامل يمتلك حساباً قديما...  


In [ ]:
import pandas as pd

#
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)


print(dormant_customers[["account_age_days", "credit_limit", "account_type", "arabic_reason"]].head(2))


    account_age_days  credit_limit  account_type  \
0               3204         20000             0   
10              1268          5000             0   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    arabic_reason  
0   يتضح من البيانات المقدمة أن العميل يعتبر "خاملًا"، وهذا قد يعود لعدة أسباب. أولاً، على الرغم من أن 

In [1]:
import os
os.getcwd()

'/content'